In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, MaxPooling3D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed
from tensorflow.keras.layers import Dropout, InputLayer
import numpy as np
import cv2
import os
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [2]:
#Get Data
# Đường dẫn đến thư mục chứa dữ liệu
base_path = 'data'  # Thay đổi thành đường dẫn thực tế đến thư mục 'data'
actions = ['up', 'right', 'left', 'down']  # Các hành động

# Khởi tạo danh sách để lưu trữ dữ liệu
data = []
labels = []

# Duyệt qua từng hành động
for action in actions:
    action_path = os.path.join(base_path, action)
    print(action_path)
    folders = os.listdir(action_path)
    
    # Duyệt qua từng folder con chứa các frame
    for folder in folders:
        folder_path = os.path.join(action_path, folder)
        images = []
        
        # Đọc 10 ảnh trong folder con
        for filename in sorted(os.listdir(folder_path))[:10]:  # Giới hạn đến 10 ảnh
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Đọc ảnh dưới dạng grayscale
            
            # Chuyển đổi kích thước ảnh nếu cần
            img = cv2.resize(img, (64, 64))  # Đảm bảo kích thước là 64x64
            
            # Thêm ảnh vào danh sách
            images.append(img.reshape((64, 64, 1)))  # Thay đổi hình dạng thành (64, 64, 1)
        
        # Thêm 10 ảnh của folder vào data và nhãn tương ứng
        if len(images) == 10:  # Chỉ thêm nếu có đủ 10 ảnh
            data.append(images)
            labels.append(action)  # Thêm nhãn hành động

# Chuyển đổi danh sách thành numpy array
data_array = np.array(data)  # Hình dạng sẽ là (n, 10, 64, 64, 1)
labels_array = np.array(labels)

# Kiểm tra kích thước dữ liệu
print(data_array.shape)  # Nên in ra (n, 10, 64, 64, 1)
print(labels_array.shape)  # Nên in ra (n,)


data\up
data\right
data\left
data\down
(1270, 10, 64, 64, 1)
(1270,)


In [3]:
label_map = {label:num for num, label in enumerate(actions)}
# print(label_map)

# X = np.expand_dims(np.array(data), axis=-1)
y = to_categorical(np.array([label_map[y] for y in labels_array]), num_classes=4)
print(y.shape)

(1270, 4)


In [4]:
# Số lượng nhãn hành động
num_classes = 4  # các nhãn hành động "up," "down," "right," "auto"
sequence_length = 10 #độ dài chuỗi video (số khung hình cho mỗi video)


model = Sequential()

# Lớp ConvLSTM2D đầu tiên
model.add(ConvLSTM2D(filters=4, kernel_size=(3, 3), activation='tanh',
                     data_format='channels_last', recurrent_dropout=0.2,
                     return_sequences=True, input_shape=(10, 64, 64, 1)))

# Bọc MaxPooling2D trong TimeDistributed
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2), padding='same')))
model.add(TimeDistributed(Dropout(0.2)))

# Lớp ConvLSTM2D thứ hai
model.add(ConvLSTM2D(filters=8, kernel_size=(3, 3), activation='tanh',
                     data_format='channels_last', recurrent_dropout=0.2,
                     return_sequences=True))

model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2), padding='same')))
model.add(TimeDistributed(Dropout(0.2)))

# Lớp ConvLSTM2D thứ ba
model.add(ConvLSTM2D(filters=14, kernel_size=(3, 3), activation='tanh',
                     data_format='channels_last', recurrent_dropout=0.2,
                     return_sequences=True))

model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2), padding='same')))
model.add(TimeDistributed(Dropout(0.2)))

# Lớp ConvLSTM2D thứ tư
model.add(ConvLSTM2D(filters=16, kernel_size=(3, 3), activation='tanh',
                     data_format='channels_last', recurrent_dropout=0.2,
                     return_sequences=False))  # Không trả về chuỗi để chuẩn bị làm phẳng

model.add(Dropout(0.2))

# Làm phẳng đầu ra để chuẩn bị cho lớp Dense
model.add(Flatten())

# Lớp Dense cuối cùng với softmax để phân loại
model.add(Dense(4, activation='softmax'))
# Hiển thị cấu trúc mô hình
model.summary()


d:\Coding\CT282 Deep Learning\Tensor\newenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_lstm2d (ConvLSTM2D)        │ (None, 10, 62, 62, 4)  │           736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 10, 31, 31, 4)  │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 10, 31, 31, 4)  │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_1 (ConvLSTM2D)      │ (None, 10, 29, 29, 8)  │         3,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 10, 15, 15, 8)  │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 10, 15, 15, 8)  │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_2 (ConvLSTM2D)      │ (None, 10, 13, 13, 14) │        11,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 10, 7, 7, 14)   │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, 10, 7, 7, 14)   │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_3 (ConvLSTM2D)      │ (None, 5, 5, 16)       │        17,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 5, 5, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │         1,604 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,316 (134.05 KB)

 Trainable params: 34,316 (134.05 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Compile mô hình
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [8]:
X_train, X_test, y_train, y_test = train_test_split(data_array, y, test_size=0.1)

print(X_train.shape)
print(X_test.shape)

(1143, 10, 64, 64, 1)
(127, 10, 64, 64, 1)


In [ ]:
# Huấn luyện mô hình
model.fit(X_train, y_train, epochs=50, validation_split=0.1 )

Epoch 1/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 17s 297ms/step - accuracy: 0.2465 - loss: 1.3881 - val_accuracy: 0.3826 - val_loss: 1.3136
Epoch 2/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 9s 263ms/step - accuracy: 0.4114 - loss: 1.2559 - val_accuracy: 0.5913 - val_loss: 1.0678
Epoch 3/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 9s 264ms/step - accuracy: 0.5581 - loss: 1.0948 - val_accuracy: 0.6174 - val_loss: 0.9176
Epoch 4/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 9s 265ms/step - accuracy: 0.5525 - loss: 1.0408 - val_accuracy: 0.6522 - val_loss: 0.8536
Epoch 5/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 9s 262ms/step - accuracy: 0.5758 - loss: 0.9752 - val_accuracy: 0.6783 - val_loss: 0.7950
Epoch 6/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 9s 261ms/step - accuracy: 0.6505 - loss: 0.8403 - val_accuracy: 0.7217 - val_loss: 0.7566
Epoch 7/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 9s 269ms/step - accuracy: 0.6760 - loss: 0.7903 - val_accuracy: 0.7304 - val_loss: 0.7215
Epoch 8/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 9s 267ms/step - accuracy: 0.6999 - loss: 0.7837 - val_accuracy: 0

In [10]:
score = model.evaluate(X_test, y_test)

print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.8714 - loss: 0.3540
Test loss:  0.37310317158699036
Test accuracy:  0.8582677245140076
